#### basic_mind
    superviskeyed learning（教師あり）: regression（回帰）classification（分類）
        keyword: 特徴量、正解データ
    unsupervised learning（教師なし）: clustering（クラスタリング）dimensionality reduction（次元削減）
    
#### 各関数：
df["].value_counts() 1列分のデータ別個数の集計  
df["].unique() 1列分のデータを重複を除いて抽出  
df.dropna(how="all" or "any", axis=0:row or 1:col)　None削除  
df.drop(index or colname, axis= 0:row or 1:col) 行またはカラム名指定削除  
df.loc[index, col] 特定の行列の抽出 :スライサOK  
df["].fillna("replaceValue")欠損値補完  
df.loc[:,from_ColName:to_ColNmae] インデックス、カラム名指定取得 条件複数指定の場合   
df_none = df[df.isnull().any(axis=1)] None行の取得  
df_if = df[(df[a]>0) & (df[y]=="val")] 行の条件取得  

df.shape　データフレームの全体概要出力  
df.columns カラム名取得


pd.concat([df_A, df_B], axis=0:addRow or 1:addCol, how=?)  
pd.merge

#### 描画
df.plot(kind="scatter", x=val, y=val) 散布図


#### 機械学習
pd.get_dummies(dataframe, drop_first=True) ダミーデータへ変換

with open("fileName",wb) as f; pickle.dump("saveModel",f) モデル保存  
with open("fileName",rb) as f; pickle.load("saveModel",f)　モデル読み込み

標準化

from sklearn.model_selection import train_test_split データの分割 複数可  
x1,x2, y1,y2 = train_test_split(df_x, df_y, test_size=0.2, random_state=0)

from sklearn import tree 決定木 分類 class_weightは任意  
model = tree.DecisionTreeClassifier(max_depth=depth, random_state=0, class_weight="balanced")  
model.fit(x,y)  
score = model.score(x=x_tr, y=y_tr)

from sklearn.linear_model import LinearRegression　回帰  
model = LinearRegression()  
model.fit(x,y) x:train y=corect  
pred_value = model.predict(x)


# 機械学習ラーニング　CHAPTER9　bank.csv

## 文字列データのダミー化

In [4]:
import pandas as pd

df = pd.read_csv("sukkiri-ml2-codes/datafiles/bank.csv")

colname = ["job","marital","education","default","housing","loan","contact"] #文字情報が含まれる列
df_str = df.loc[:,colname] #対象列のみ取得
df_dummy = pd.get_dummies(df_str, drop_first=True) #文字情報のデータを数値化
# Drop the original categorical columns from df
df_without_cat = df.drop(colname, axis=1) #文字列情報が含まれる列を削除
# Concatenate the numerical columns with dummy variables horizontally

df_all = pd.concat([df, df_dummy], axis=1) #文字列、ダミーデータ含む全データ
df2 = pd.concat([df_without_cat, df_dummy], axis=1)#文字列データをダミーデータ化した新しいＤＦ

## 訓練データとテストデータの分割、不要な項目を削除

In [7]:
from sklearn.model_selection import train_test_split
df2 = df2.drop(["id","age","amount","day","month"],axis=1)#訓練データに関係のなさそうな列を削除
train_val, test = train_test_split(df2,test_size=0.2,random_state=0)#訓練データとテストデータに分割

## 機械学習による予測値の設定

In [10]:
# 機械学習による予測値
from sklearn.linear_model import LinearRegression
# need to predict
pred_x = train_val[train_val.isnull().any(axis=1)] #欠損値があるデータ
pred_tr_val = train_val.dropna() #欠損値のない訓練データ


In [12]:
x = pred_tr_val.drop(["duration"],axis=1) #訓練データの列
y = pred_tr_val["duration"] #正解データの列
model = LinearRegression()
model.fit(x,y)#学習！！

pred_x = pred_x.drop(["duration"],axis=1) #欠損値を予測するための検証データ
pred_y = model.predict(pred_x)#予測値取得

train_val.loc[train_val["duration"].isnull(), "duration"] = pred_y#もとになる訓練データに欠損値を設定する

## データの標準化

In [15]:
## 性能テスト

from sklearn import tree
from sklearn.preprocessing import StandardScaler
def learn(x,t,depth=1): # 訓練データと検証データのスコアを返す処理
    x_tr, x_test, y_tr, y_test = train_test_split(x, t, test_size=0.2, random_state=0)

    # 課題・・・・・・・・・・・・・・・・・・・・・・・・・・・・・・
    # 訓練データの標準化
    sc_model_x = StandardScaler()
    sc_model_y = StandardScaler()
    sc_model_x.fit(x_tr) #標準化する前準備
    sc_model_y.fit(y_tr)
    
    sc_x_tr = sc_model_x.transform(x_tr)#データの標準化処理
    sc_y_tr = sc_model_y.transform(y_tr)

    # x_tr = sc_model_x.inverse_tranceform(sc_x_tr)#標準化されたデータをもとに戻す処理　predictなどで予測値を取得する際はもとに戻す必要あり　
    
    model = tree.DecisionTreeClassifier(max_depth=depth, random_state=0, class_weight="balanced")
    # model = tree.DecisionTreeClassifier(max_depth=depth, random_state=0)
    model.fit(x_tr, y_tr)#学習モデルの訓練データは標準化されたデータで学習されていることに注意
    
    # 検証データの標準化
    sc_x_val = sc_model_x.transform(x_test)#データの標準化処理
    sc_y_val = sc_model_y.transform(y_test)

    score = model.score(x_tr, y_tr)#訓練データのスコア
    score2 = model.score(x_test, y_test)#検証データのスコア
    return round(score,3), round(score2,3), model

x = train_val.drop("y",axis=1)
t = train_val[["y"]]

for i in range(1,15):
    tr_score, test_score, model = learn(x, t, i)
    print(f"深さ{i}：訓練スコア{tr_score} テストスコア{test_score}")

# 性能が高いモデルでテスト
tr_score, test_score, model = learn(x, t, 8)
_test = test.dropna()
x = _test.drop("y",axis=1)
y = _test["y"]

score = model.score(x, y)
print(f"実績スコア{round(score,3)}")

深さ1：訓練スコア0.757 テストスコア0.756
深さ2：訓練スコア0.757 テストスコア0.756
深さ3：訓練スコア0.712 テストスコア0.706
深さ4：訓練スコア0.753 テストスコア0.746
深さ5：訓練スコア0.787 テストスコア0.779
深さ6：訓練スコア0.781 テストスコア0.772
深さ7：訓練スコア0.799 テストスコア0.788
深さ8：訓練スコア0.812 テストスコア0.797
深さ9：訓練スコア0.833 テストスコア0.801
深さ10：訓練スコア0.842 テストスコア0.793
深さ11：訓練スコア0.86 テストスコア0.795
深さ12：訓練スコア0.871 テストスコア0.794
深さ13：訓練スコア0.889 テストスコア0.795
深さ14：訓練スコア0.901 テストスコア0.79
実績スコア0.719


## マハラノビスによる外れ値の除去

In [18]:
from sklearn.covariance import MinCovDet
mcd = MinCovDet(random_state=0, support_fraction=0.7)

train_val = train_val.reset_index()
mcd.fit(train_val)#マハラノビス距離を計算するために必要な共分散行列を計算

#!!point!! 元データを使って距離を計測しているがシリーズに変換した際、indexが再度採番されていることに注意
#対象方法としては、元データのインデックスリセットするなどの処理が必要
distance = mcd.mahalanobis(train_val) #距離測定
s_distance = pd.Series(distance)#describe関数を使用するためにシリーズに変換

tmp=s_distance.describe()

iqr = tmp["75%"] - tmp["25%"] #ＩＱＲ＝四分位範囲＝第3四分位数（75％）－第1四分位（25％）
maxval = (iqr*1.5) + tmp["75%"]#上限値のしきい値=第3四分位数＋（1.5*75％）
minval = tmp["25%"] - (iqr*1.5)#下限値のしきい値=第1四分位数－（1.5*75％）

clear_data = s_distance[(s_distance > maxval) | (minval > s_distance)]



In [33]:
train_val_without_mahalanobis = train_val.drop(clear_data.index)
print("train_val",train_val.shape)
print("s_distance",s_distance.shape)
print("clear_data",clear_data.shape)
print("train_val_without_mahalanobis",train_val2.shape)



train_val (21702, 26)
s_distance (21702,)
clear_data (2620,)
train_val_without_mahalanobis (19082, 26)


In [28]:
# 1. 相関係数を計算
# yと他の特徴量との相関係数を計算し、絶対値で降順にソート
correlations = train_val.corr()['y'].abs().sort_values(ascending=False)
print("相関係数（上位10項目）:")
print(correlations[1:11])  # yとの相関を除いて上位10項目を表示


# # 相関係数上位5項目についてボックスプロットを作成
# top_features = correlations[1:6].index
# fig, axes = plt.subplots(2, 3, figsize=(15, 10))
# axes = axes.ravel()

# for idx, feature in enumerate(top_features):
#     sns.boxplot(data=train_val, x='y', y=feature, ax=axes[idx])
#     axes[idx].set_title(f'{feature} vs y')

# plt.tight_layout()
# plt.show()

相関係数（上位10項目）:
duration                     0.453555
housing_yes                  0.197832
campaign                     0.146778
contact_sending _document    0.131849
loan_yes                     0.096758
marital_single               0.066403
marital_married              0.055259
job_student                  0.051178
job_blue-collar              0.039722
job_unemployed               0.030056
Name: y, dtype: float64
相関係数（上位10項目）:
duration                     0.458181
housing_yes                  0.209083
campaign                     0.192961
contact_sending _document    0.152239
loan_yes                     0.107319
marital_single               0.068297
marital_married              0.055134
job_student                  0.054618
job_blue-collar              0.045171
education_tertiary           0.030686
Name: y, dtype: float64
